# 2Homez Methodology
### A closer look at the datasets and code that make 2Homez work
Data found here: http://insideairbnb.com/get-the-data.html

## Part I: Airbnb Data, San Francisco
### Airbnb listings dataset
We use insideairbnb.com to get our two historical airbnb datasets. The first dataset is a detailed archive of all listings in the San Francisco area. The second dataset is a calendar archive of all listings and their price and availability on supplied dates. We use these two datasets to create a model that can extrapolate seasonal airbnb rents for homes not found in the archive datasets.

In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# First we read in the data from the csv
SF_data = pd.read_csv("airbnb_data/sf_listings.csv")
SF_data.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,11187767,https://www.airbnb.com/rooms/11187767,20160702162156,2016-07-02,Huge Seacliff Penthouse With Views,Amazing views from this immense 3 bedroom/ 2 b...,"Spacious patio deck, library with over 300 mov...",Amazing views from this immense 3 bedroom/ 2 b...,none,"The neighborhood includes Legion of Honor, Sut...",...,8.0,t,NaN,SAN FRANCISCO,t,strict,f,f,1,0.88
1,6938818,https://www.airbnb.com/rooms/6938818,20160702162156,2016-07-02,Best Secret in Town,My two story house is located in the quite sid...,The room is spacious and it is on the top leve...,My two story house is located in the quite sid...,none,It is in the city and close to everything. Par...,...,9.0,t,S. F. Short-Term Residential Rental Registrati...,SAN FRANCISCO,f,strict,f,f,2,1.85


There are a lot of columns that we find unnecessary for our use case; we drop these. We assume that 2Homez clients will rent the entire property they find.

In [142]:
SF_data = SF_data[['id', 'neighbourhood_cleansed', 'latitude','longitude','bathrooms','bedrooms','room_type','price']]
SF_data = SF_data[SF_data['room_type'] == 'Entire home/apt']
SF_data.drop(['room_type'], axis=1, inplace=True)
SF_data.head(2)

,id,neighbourhood_cleansed,latitude,longitude,bathrooms,bedrooms,price
0,11187767,Seacliff,37.779685,-122.510472,2.0,3.0,$400.00
2,9395222,Seacliff,37.781433,-122.505179,1.0,1.0,$155.00


### Exploration of Airbnb listings dataset by neighborhood

In [91]:
SF_data['neighbourhood_cleansed'].value_counts()[:5]

Mission                554
Western Addition       471
South of Market        354
Castro/Upper Market    327
Bernal Heights         277
Name: neighbourhood_cleansed, dtype: int64

There are high numbers of listings in Mission and Western Addition.

In [149]:
SF_data.price = SF_data.price.replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)
avg_neigh_price = SF_data.groupby('neighbourhood_cleansed').\
                          aggregate({'price' : np.mean}).\
                          rename(index=str, columns={"price": "avg_price"}).\
                          sort_values(by='avg_price', ascending=0)
                                                    

SF_data = SF_data.join(avg_neigh_price, on='neighbourhood_cleansed')
SF_data.head()

,id,neighbourhood_cleansed,latitude,longitude,bathrooms,bedrooms,price,avg_price
0,11187767,Seacliff,37.779685,-122.510472,2.0,3.0,400.0,287.909091
2,9395222,Seacliff,37.781433,-122.505179,1.0,1.0,155.0,287.909091
3,8388658,Seacliff,37.787664,-122.489152,2.5,3.0,895.0,287.909091
4,7856443,Seacliff,37.782133,-122.492730,2.5,3.0,195.0,287.909091
5,4781448,Seacliff,37.781797,-122.492492,2.5,3.0,395.0,287.909091


Here we can see the five neighborhoods in San Francisco with the highest average rent prices present in our archive data. We also need one hot encodings of the neighborhood to perform nearest neighbors when classifying out-of-archive houses into neighborhoods found in-archive.

In [150]:
SF_data_ohe = pd.get_dummies(SF_data['neighbourhood_cleansed'])
SF_data_ohe = SF_data.join(SF_data_ohe)
SF_data_ohe.drop(['neighbourhood_cleansed','price'],axis=1,inplace=True)
ll2nn = SF_data[['latitude','longitude','neighbourhood_cleansed']]

### Airbnb calendar archive dataset
We now need to use the calendar dataset to calculate seasonal rental prices for each listing.

In [64]:
sf_avail = pd.read_csv("airbnb_data/sf_calendar.csv")
sf_avail.head()

,listing_id,date,available,price
0,11187767,2017-03-12,f,NaN
1,11187767,2017-03-11,f,NaN
2,11187767,2017-03-10,f,NaN
3,11187767,2017-03-09,f,NaN
4,11187767,2017-03-08,f,NaN


Each entry in the dataset has a date associated with it; we need to turn these dates into seasons in order to calculate seasonal properties of each listing.

In [65]:
findSeason = {1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',6:'Summer',7:'Summer',\
             8:'Summer',9:'Fall',10:'Fall',11:'Fall',12:'Winter'}
sf_avail['date'] = pd.to_datetime(sf_avail['date'])
sf_avail['Season'] = sf_avail['date'].apply(lambda x: findSeason[x.month])

### Using calendar dataset to compute seasonal rental prices
Using the brand new season column, we now proceed to calculating seasonal average rental prices for each listing.

In [66]:
sf_prices = sf_avail.dropna().drop(['date','available'],axis=1)
sf_prices['price'] = sf_prices['price'].replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)
sf_prices = pd.pivot_table(sf_prices,index=['listing_id'], columns=['Season'],aggfunc=np.mean)
sf_prices.head()

price                                    
Season            Fall      Spring      Summer      Winter
listing_id                                                
958         171.560000  171.695652  171.783784  171.733333
5193        175.000000  160.000000  167.195122  161.518987
5841        183.432432  183.543478  183.903226  183.600000
5858        250.000000  250.000000  250.000000  250.000000
7918         65.000000   65.000000   65.000000   65.000000

### Prepare training and test data for price model
Now that we have separate dataframes for each listing's seasonal availability and each listing's seasonal rent prices, we must join this information with our one hot encoding dataframe in order to create data to train regressions on.

In [192]:
sf_total = SF_data_ohe.set_index(['id'])
# sf_total = SF_data.set_index(['id']).drop(['price','neighbourhood_cleansed'],axis=1)
sf_total.rename(columns={'id':'listing_id'},inplace=True)
sf_total = sf_total.join(sf_prices)
sf_total.rename(columns={i: i[1] + i[0] for i in sf_total.columns if type(i) == tuple},inplace=True)
labels = ['Fallprice','Springprice','Summerprice','Winterprice']
sf_labels = sf_total[labels]
sf_train = sf_total.drop(labels, axis=1)
sf_labels = sf_labels.T.fillna(sf_labels.mean(axis=1)).T.dropna()
sf_train[labels] = sf_labels
sf_train = sf_train.dropna()
sf_labels = sf_train[labels]
sf_train = sf_train.drop(labels,axis=1)
sf_train.drop(['latitude','longitude'],axis=1, inplace=True)
sf_train.head()

,bathrooms,bedrooms,avg_price,Bayview,Bernal Heights,Castro/Upper Market,Chinatown,Crocker Amazon,Diamond Heights,Downtown/Civic Center,...,Presidio,Presidio Heights,Russian Hill,Seacliff,South of Market,Treasure Island/YBI,Twin Peaks,Visitacion Valley,West of Twin Peaks,Western Addition
id,,,,,,,,,,,,,,,,,,,,,
11187767,2.0,3.0,287.909091,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9395222,1.0,1.0,287.909091,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7856443,2.5,3.0,287.909091,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4781448,2.5,3.0,287.909091,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1427660,1.0,1.0,287.909091,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


This is now what our training data looks like. Here is what the labels for these data points will look like:

In [193]:
sf_labels.head()

,Fallprice,Springprice,Summerprice,Winterprice
id,,,,
11187767,489.772727,501.641414,515.151515,500.000000
9395222,185.052632,166.304348,166.891892,172.749624
7856443,513.333333,456.666667,400.000000,456.666667
4781448,327.000000,327.000000,327.000000,327.000000
1427660,150.000000,150.000000,150.000000,150.000000


We will now partition our data in order to have a validation set for regression model accuracy testing:

In [194]:
model_train, model_label = sf_train[:-634], sf_labels[:-634]
val_train, val_label = sf_train[-634:], sf_labels[-634:]

## Part II: Regression Models on Airbnb data
We can now use this training data to predict seasonal rent prices for out-of-archive houses, namely houses for sale on Zillow.

#### Linear Regression

In [195]:
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, r2_score
regr = linear_model.RidgeCV()
regr.fit(model_train,model_label)
pred = regr.predict(val_train)
print("MAE:", mean_absolute_error(pred,val_label))
print("R2:", r2_score(pred,val_label,multioutput='uniform_average'))
print("Variance score:", regr.score(val_train,val_label))
compare = pd.DataFrame()
compare[['FA','SP','SU','WI']] = val_label
compare[['FA_Pred','SP_Pred','SU_Pred','WI_Pred']] = \
pd.DataFrame(pred,index=compare.index)
compare.head(10)

MAE: 131.861574827
R2: -3.08353523108
Variance score: 0.305440069567


,FA,SP,SU,WI,FA_Pred,SP_Pred,SU_Pred,WI_Pred
id,,,,,,,,
13688776,114.285714,114.285714,114.285714,114.285714,102.919550,109.408802,112.019010,104.782775
12512343,156.000000,156.000000,156.000000,156.000000,102.919550,109.408802,112.019010,104.782775
13318241,160.000000,160.000000,160.000000,160.000000,240.130727,242.205995,243.844055,236.938491
864265,266.813187,263.478261,266.029412,260.159091,240.130727,242.205995,243.844055,236.938491
12004930,139.000000,139.000000,139.000000,139.000000,240.130727,242.205995,243.844055,236.938491
10840679,154.473684,154.239130,154.852941,154.333333,240.130727,242.205995,243.844055,236.938491
9464133,375.000000,375.000000,375.000000,375.000000,240.130727,242.205995,243.844055,236.938491
13641752,200.000000,205.000000,210.000000,205.000000,240.130727,242.205995,243.844055,236.938491
12854741,237.758621,229.106583,220.454545,229.106583,240.130727,242.205995,243.844055,236.938491


#### Gradient Boosting Regressor

In [208]:
from sklearn import ensemble
from sklearn.multioutput import MultiOutputRegressor
nes = np.linspace(1,70,70).astype(int)
best_score,best_ne = 0,0
for ne in nes:
    gbr = MultiOutputRegressor(ensemble.GradientBoostingRegressor(n_estimators=ne))
    gbr.fit(model_train,model_label)
    pred = gbr.predict(val_train)
    score = gbr.score(val_train,val_label)
    if score > best_score:
        best_score,best_ne = score, ne
gbr = MultiOutputRegressor(ensemble.GradientBoostingRegressor(n_estimators=best_ne))
gbr.fit(model_train,model_label)
pred = gbr.predict(val_train)
score = gbr.score(val_train,val_label)
print("MAE:", mean_absolute_error(pred,val_label))
print("R2:", r2_score(pred,val_label,multioutput='uniform_average'))
print("Variance score:", score)
compare = pd.DataFrame()
compare[['FA','SP','SU','WI']] = val_label
compare[['FA_Pred','SP_Pred','SU_Pred','WI_Pred']] = \
pd.DataFrame(pred,index=compare.index)
compare.head(10)

MAE: 126.883729433
R2: 0.455204216053
Variance score: 0.64762686857


,FA,SP,SU,WI,FA_Pred,SP_Pred,SU_Pred,WI_Pred
id,,,,,,,,
13688776,114.285714,114.285714,114.285714,114.285714,261.297341,262.134581,261.705397,256.632197
12512343,156.000000,156.000000,156.000000,156.000000,261.297341,262.134581,261.705397,256.632197
13318241,160.000000,160.000000,160.000000,160.000000,261.297341,262.134581,261.705397,256.632197
864265,266.813187,263.478261,266.029412,260.159091,261.297341,262.134581,261.705397,256.632197
12004930,139.000000,139.000000,139.000000,139.000000,261.297341,262.134581,261.705397,256.632197
10840679,154.473684,154.239130,154.852941,154.333333,261.297341,262.134581,261.705397,256.632197
9464133,375.000000,375.000000,375.000000,375.000000,261.297341,262.134581,261.705397,256.632197
13641752,200.000000,205.000000,210.000000,205.000000,261.297341,262.134581,261.705397,256.632197
12854741,237.758621,229.106583,220.454545,229.106583,261.297341,262.134581,261.705397,256.632197


#### Random Forest Regressor

In [207]:
from sklearn import ensemble
from sklearn.multioutput import MultiOutputRegressor
nes = np.linspace(1,70,70).astype(int)
best_score,best_ne = 0,0
for ne in nes:
    rfr = MultiOutputRegressor(ensemble.RandomForestRegressor(n_estimators=ne))
    rfr.fit(model_train,model_label)
    pred = rfr.predict(val_train)
    score = rfr.score(val_train,val_label)
    if score > best_score:
        best_score,best_ne = score, ne
rfr = MultiOutputRegressor(ensemble.RandomForestRegressor(n_estimators=best_ne))
rfr.fit(model_train,model_label)
pred = rfr.predict(val_train)
score = rfr.score(val_train,val_label)
print("MAE:", mean_absolute_error(pred,val_label))
print("R2:", r2_score(pred,val_label,multioutput='uniform_average'))
print("Variance score:", score)
compare = pd.DataFrame()
compare[['FA','SP','SU','WI']] = val_label
compare[['FA_Pred','SP_Pred','SU_Pred','WI_Pred']] = \
pd.DataFrame(pred,index=compare.index)
compare.head(10)

MAE: 110.613803609
R2: 0.565054705655
Variance score: 0.656538910322


,FA,SP,SU,WI,FA_Pred,SP_Pred,SU_Pred,WI_Pred
id,,,,,,,,
13688776,114.285714,114.285714,114.285714,114.285714,204.046682,219.248114,220.806106,205.956999
12512343,156.000000,156.000000,156.000000,156.000000,204.046682,219.248114,220.806106,205.956999
13318241,160.000000,160.000000,160.000000,160.000000,214.862824,209.031045,218.340282,215.761672
864265,266.813187,263.478261,266.029412,260.159091,214.862824,209.031045,218.340282,215.761672
12004930,139.000000,139.000000,139.000000,139.000000,214.862824,209.031045,218.340282,215.761672
10840679,154.473684,154.239130,154.852941,154.333333,214.862824,209.031045,218.340282,215.761672
9464133,375.000000,375.000000,375.000000,375.000000,214.862824,209.031045,218.340282,215.761672
13641752,200.000000,205.000000,210.000000,205.000000,214.862824,209.031045,218.340282,215.761672
12854741,237.758621,229.106583,220.454545,229.106583,214.862824,209.031045,218.340282,215.761672


#### Extra Trees Regressor

In [205]:
from sklearn import ensemble
from sklearn.multioutput import MultiOutputRegressor
nes = np.linspace(1,70,70).astype(int)
best_score,best_ne = 0,0
for ne in nes:
    etr = MultiOutputRegressor(ensemble.ExtraTreesRegressor(n_estimators=ne))
    etr.fit(model_train,model_label)
    pred = etr.predict(val_train)
    score = etr.score(val_train,val_label)
    if score > best_score:
        best_score,best_ne = score, ne
etr = MultiOutputRegressor(ensemble.ExtraTreesRegressor(n_estimators=best_ne))
etr.fit(model_train,model_label)
pred = etr.predict(val_train)
score = etr.score(val_train,val_label)
print("MAE:", mean_absolute_error(pred,val_label))
print("R2:", r2_score(pred,val_label,multioutput='uniform_average'))
print("Variance score:", score)
compare = pd.DataFrame()
compare[['FA','SP','SU','WI']] = val_label
compare[['FA_Pred','SP_Pred','SU_Pred','WI_Pred']] = \
pd.DataFrame(pred,index=compare.index)
compare.head(10)

MAE: 108.164934005
R2: 0.455972047419
Variance score: 0.664222682178


,FA,SP,SU,WI,FA_Pred,SP_Pred,SU_Pred,WI_Pred
id,,,,,,,,
13688776,114.285714,114.285714,114.285714,114.285714,207.058790,221.572381,224.700421,205.545642
12512343,156.000000,156.000000,156.000000,156.000000,207.058790,221.572381,224.700421,205.545642
13318241,160.000000,160.000000,160.000000,160.000000,215.960981,212.349706,213.618566,213.454549
864265,266.813187,263.478261,266.029412,260.159091,215.960981,212.349706,213.618566,213.454549
12004930,139.000000,139.000000,139.000000,139.000000,215.960981,212.349706,213.618566,213.454549
10840679,154.473684,154.239130,154.852941,154.333333,215.960981,212.349706,213.618566,213.454549
9464133,375.000000,375.000000,375.000000,375.000000,215.960981,212.349706,213.618566,213.454549
13641752,200.000000,205.000000,210.000000,205.000000,215.960981,212.349706,213.618566,213.454549
12854741,237.758621,229.106583,220.454545,229.106583,215.960981,212.349706,213.618566,213.454549


#### Adaboost Regressor

In [206]:
from sklearn import ensemble
from sklearn.multioutput import MultiOutputRegressor
nes = np.linspace(1,70,70).astype(int)
best_score,best_ne = 0,0
for ne in nes:
    abr = MultiOutputRegressor(ensemble.AdaBoostRegressor(n_estimators=ne))
    abr.fit(model_train,model_label)
    pred = abr.predict(val_train)
    score = abr.score(val_train,val_label)
    if score > best_score:
        best_score,best_ne = score, ne
abr = MultiOutputRegressor(ensemble.AdaBoostRegressor(n_estimators=best_ne))
abr.fit(model_train,model_label)
pred = abr.predict(val_train)
score = abr.score(val_train,val_label)
print("MAE:", mean_absolute_error(pred,val_label))
print("R2:", r2_score(pred,val_label,multioutput='uniform_average'))
print("Variance score:", score)
compare = pd.DataFrame()
compare[['FA','SP','SU','WI']] = val_label
compare[['FA_Pred','SP_Pred','SU_Pred','WI_Pred']] = \
pd.DataFrame(pred,index=compare.index)
compare.head(10)

MAE: 133.490574197
R2: -6.36015830434
Variance score: 0.266160557795


,FA,SP,SU,WI,FA_Pred,SP_Pred,SU_Pred,WI_Pred
id,,,,,,,,
13688776,114.285714,114.285714,114.285714,114.285714,243.490224,227.387272,244.048342,228.603152
12512343,156.000000,156.000000,156.000000,156.000000,243.490224,227.387272,244.048342,228.603152
13318241,160.000000,160.000000,160.000000,160.000000,243.490224,227.387272,244.048342,228.603152
864265,266.813187,263.478261,266.029412,260.159091,243.490224,227.387272,244.048342,228.603152
12004930,139.000000,139.000000,139.000000,139.000000,243.490224,227.387272,244.048342,228.603152
10840679,154.473684,154.239130,154.852941,154.333333,243.490224,227.387272,244.048342,228.603152
9464133,375.000000,375.000000,375.000000,375.000000,243.490224,227.387272,244.048342,228.603152
13641752,200.000000,205.000000,210.000000,205.000000,243.490224,227.387272,244.048342,228.603152
12854741,237.758621,229.106583,220.454545,229.106583,243.490224,227.387272,244.048342,228.603152


## Part III: Using Zillow Data
### Classifying Zillow latitude,longitude into neighborhoods
We now have a model that can predict seasonal rental prices given details about a house. We also have access to a csv file of homes for sale posted on Zillow from a recent date. We will now need to reconcile the two objects we have so that we can use our model on Zillow listings. For this, we need to associate each Zillow listing with a neighborhood. We do so using our ll2nn dataframe, creating a nearest neighbors model to match each Zillow home with its closest partners found in our Airbnb data.

In [294]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
zillow_ll = pd.read_csv("sf_zillow_apr_23_w_latlon_w_photo_url.csv")
latlon, nbrhd = shuffle(ll2nn[['latitude','longitude']],ll2nn['neighbourhood_cleansed'])
ll2nn_model_train, ll2nn_model_label = latlon[:-100], nbrhd[:-100]
ll2nn_val_train, ll2nn_val_label = latlon[-100:], nbrhd[-100:]
ll2nn_clf = KNeighborsClassifier(n_neighbors=2)
ll2nn_clf.fit(ll2nn_model_train, ll2nn_model_label)
pred = ll2nn_clf.predict(ll2nn_val_train)
print("Accuracy of (lat, lon)-> neighborhood clf:", accuracy_score(ll2nn_val_label, pred))
zillow_ll['neighborhood'] = ll2nn_clf.predict(zillow_ll[['latitude','longitude']])
zillow_ll.head()

Accuracy of (lat, lon)-> neighborhood clf: 0.94


,Unnamed: 0,Unnamed: 0.1,address,city,state,zip,price,sqft,bedrooms,bathrooms,days_on_zillow,sale_type,url,latitude,longitude,photo_url,neighborhood
0,0,0,181 Ofarrell St Ste 305,San Francisco,CA,94102.0,1040000.0,1334.0,1.0,2.0,NaN,Condo For Sale,http://www.zillow.com/homes/for_sale/82785263_...,37.786274,-122.407564,https://photos.zillowstatic.com/p_h/ISqtn3q6r2...,Downtown/Civic Center
1,1,1,342 Hayes St Apt B,San Francisco,CA,94102.0,1269000.0,NaN,2.0,2.5,NaN,Condo For Sale,http://www.zillow.com/homes/for_sale/15078395_...,37.777161,-122.422158,https://photos.zillowstatic.com/p_h/ISqtjgl08c...,Downtown/Civic Center
2,2,2,650 Turk St Unit 406,San Francisco,CA,94102.0,499900.0,398.0,0.0,1.0,NaN,Condo For Sale,http://www.zillow.com/homes/for_sale/209988348...,37.782140,-122.420023,https://photos.zillowstatic.com/p_h/ISa5asnpms...,Downtown/Civic Center
3,3,3,181 Ofarrell St Ste 509,San Francisco,CA,94102.0,948000.0,1076.0,1.0,2.0,NaN,Condo For Sale,http://www.zillow.com/homes/for_sale/82785281_...,37.786274,-122.407564,https://photos.zillowstatic.com/p_h/ISqpgbcud7...,Downtown/Civic Center
4,4,4,653 Fell St,San Francisco,CA,94102.0,1298000.0,1760.0,0.0,NaN,NaN,Apartment For Sale,http://www.zillow.com/homes/for_sale/209463014...,37.774949,-122.428593,https://photos.zillowstatic.com/p_h/IS62dtea2h...,Western Addition


### Using airbnb-derived price regression model on Zillow data
We now will take our Zillow dataframe and perform the necessary transformations required to input it into our regression model.

In [295]:
zillow_ll = zillow_ll[zillow_ll['bedrooms'] > 0]
zillow_ll = zillow_ll[zillow_ll['bathrooms'] > 0]
zillow_ll.set_index('address', inplace=True)
zillow_price = zillow_ll.drop(['Unnamed: 0', 'Unnamed: 0.1', 'city', 'state', 'zip', 'price', 'sqft',
                              'days_on_zillow','sale_type', 'url', 'latitude', 'longitude','photo_url'],axis=1)

zillow_price.head()

,bedrooms,bathrooms,neighborhood
address,,,
181 Ofarrell St Ste 305,1.0,2.0,Downtown/Civic Center
342 Hayes St Apt B,2.0,2.5,Downtown/Civic Center
181 Ofarrell St Ste 509,1.0,2.0,Downtown/Civic Center
294 Page St,4.0,4.0,Western Addition
24 Page St Unit 5,3.0,3.0,Downtown/Civic Center


In [296]:
zillow_price.rename(columns={'neighborhood':'neighbourhood_cleansed'},inplace=True)
zillow_price = zillow_price.join(avg_neigh_price, on='neighbourhood_cleansed')
zillow_price_ohe = pd.get_dummies(zillow_price['neighbourhood_cleansed'])
zillow_price_ohe = zillow_price.join(zillow_price_ohe).drop(['neighbourhood_cleansed'],axis=1)
# zillow_price_ohe = zillow_price_ohe.T.fillna(zillow_price_ohe.mean(axis=1)).T
zillow_price_ohe.head(4)

,bedrooms,bathrooms,avg_price,Bayview,Bernal Heights,Castro/Upper Market,Chinatown,Crocker Amazon,Diamond Heights,Downtown/Civic Center,...,Parkside,Potrero Hill,Presidio Heights,Russian Hill,Seacliff,South of Market,Twin Peaks,Visitacion Valley,West of Twin Peaks,Western Addition
address,,,,,,,,,,,,,,,,,,,,,
181 Ofarrell St Ste 305,1.0,2.0,257.578544,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
342 Hayes St Apt B,2.0,2.5,257.578544,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
181 Ofarrell St Ste 509,1.0,2.0,257.578544,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
294 Page St,4.0,4.0,363.569002,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [297]:
zillow_price_ohe['Presidio'] = 0
zillow_price_ohe['Treasure Island/YBI'] = 0
pred = etr.predict(zillow_price_ohe)
pred

array([[ 272.03733333,  267.19710145,  267.47692308,  268.75972222],
       [ 454.81495311,  430.67496491,  394.01348127,  439.68488839],
       [ 272.03733333,  267.19710145,  267.47692308,  268.75972222],
       ..., 
       [ 249.52749307,  235.59748133,  253.42128983,  232.41557603],
       [ 249.52749307,  235.59748133,  253.42128983,  232.41557603],
       [ 249.52749307,  235.59748133,  253.42128983,  232.41557603]])

In [302]:
pred_df = pd.DataFrame(pred, index = zillow_ll.index)
pred_df.columns = ['Fall Pred Price', 'Spring Pred Price', 'Summer Pred Price', 'Winter Pred Price']
zillow_final = zillow_ll.join(pred_df)
zillow_final = zillow_final.reset_index()
zillow_final.head()

,address,Unnamed: 0,Unnamed: 0.1,city,state,zip,price,sqft,bedrooms,bathrooms,...,sale_type,url,latitude,longitude,photo_url,neighborhood,Fall Pred Price,Spring Pred Price,Summer Pred Price,Winter Pred Price
0,181 Ofarrell St Ste 305,0,0,San Francisco,CA,94102.0,1040000.0,1334.0,1.0,2.0,...,Condo For Sale,http://www.zillow.com/homes/for_sale/82785263_...,37.786274,-122.407564,https://photos.zillowstatic.com/p_h/ISqtn3q6r2...,Downtown/Civic Center,272.037333,267.197101,267.476923,268.759722
1,342 Hayes St Apt B,1,1,San Francisco,CA,94102.0,1269000.0,NaN,2.0,2.5,...,Condo For Sale,http://www.zillow.com/homes/for_sale/15078395_...,37.777161,-122.422158,https://photos.zillowstatic.com/p_h/ISqtjgl08c...,Downtown/Civic Center,454.814953,430.674965,394.013481,439.684888
2,181 Ofarrell St Ste 509,3,3,San Francisco,CA,94102.0,948000.0,1076.0,1.0,2.0,...,Condo For Sale,http://www.zillow.com/homes/for_sale/82785281_...,37.786274,-122.407564,https://photos.zillowstatic.com/p_h/ISqpgbcud7...,Downtown/Civic Center,272.037333,267.197101,267.476923,268.759722
3,294 Page St,6,6,San Francisco,CA,94102.0,5195000.0,6950.0,4.0,4.0,...,House For Sale,http://www.zillow.com/homes/for_sale/15078760_...,37.773943,-122.425587,https://photos.zillowstatic.com/p_h/ISax752dvb...,Western Addition,784.102288,676.957830,704.469084,670.828033
4,24 Page St Unit 5,7,7,San Francisco,CA,94102.0,1495000.0,1740.0,3.0,3.0,...,Condo For Sale,http://www.zillow.com/homes/for_sale/15078709_...,37.774480,-122.421362,https://photos.zillowstatic.com/p_h/ISe8vxtlzq...,Downtown/Civic Center,372.945468,348.716025,342.551475,344.832310


These results are then written to csv in order to populate our website's offerings!

In [301]:
zillow_final.to_csv("zillow_full_with_rental_estimates.csv")